In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.merge import merge
from rasterio.transform import rowcol
from pyproj import Transformer
import pandas as pd
from glob import glob

# --- 1) Input parameters ---
tif_folder = '/Users/veronicaage/Downloads/datasetsA/rainier_2007/dtm'   # folder containing your .tif files
csv_path   = '/Users/veronicaage/Downloads/mora-das.csv'                 # CSV with columns Channel, Latitude [°], Longitude [°]
buffer_m   = 1000   # buffer in meters around the points

# --- 2) Read and clean the CSV ---
df = pd.read_csv(csv_path)
df = df.dropna(subset=['Channel', 'Latitude [°]', 'Longitude [°]'], how='any')

# --- 3) Project GPS coordinates to EPSG:2927 ---
transformer = Transformer.from_crs("EPSG:4326", "EPSG:2927", always_xy=True)
x_pts, y_pts = transformer.transform(df['Longitude [°]'].values,
                                     df['Latitude [°]'].values)

# --- 4) Create bounding box with buffer ---
minx, maxx = x_pts.min() - buffer_m, x_pts.max() + buffer_m
miny, maxy = y_pts.min() - buffer_m, y_pts.max() + buffer_m

# --- 5) Open and filter only tiles that intersect the bounding box ---
tif_files = glob(f"{tif_folder}/*.tif")
src_files = [rasterio.open(fp) for fp in tif_files]
overlapping = []
for src in src_files:
    left, bottom, right, top = src.bounds
    if not (right < minx or left > maxx or top < miny or bottom > maxy):
        overlapping.append(src)
if not overlapping:
    raise RuntimeError("No DTM tile intersects the area of interest.")

# --- 6) Merge only the intersecting region (at native resolution) ---
res_x = abs(overlapping[0].transform.a)
res_y = abs(overlapping[0].transform.e)
mosaic, out_trans = merge(
    overlapping,
    method='max',
    bounds=(minx, miny, maxx, maxy),
    res=(res_x, res_y)
)

# Close all datasets
for src in src_files:
    src.close()

# --- 7) Function to compute slope in degrees ---
def compute_slope(dem, transform):
    dx = abs(transform.a)
    dy = abs(transform.e)
    dz_dx = np.gradient(dem, axis=1) / dx
    dz_dy = np.gradient(dem, axis=0) / dy
    slope_rad = np.arctan(np.sqrt(dz_dx**2 + dz_dy**2))
    return np.degrees(slope_rad)

# --- 8) Compute slope from the cropped DEM ---
dem_crop   = mosaic[0]
slope_crop = compute_slope(dem_crop, out_trans)

# --- 9) Sample slope at each GPS point ---
rows, cols = rowcol(out_trans, x_pts, y_pts)
slope_pts = np.full(len(x_pts), np.nan, dtype=float)
h, w = slope_crop.shape
for i, (r, c) in enumerate(zip(rows, cols)):
    if 0 <= r < h and 0 <= c < w:
        slope_pts[i] = slope_crop[r, c]

df['slope_deg'] = slope_pts
output_csv = '/Users/veronicaage/Downloads/mora-das-slope.csv'
df.to_csv(output_csv, index=False)
print(f"Slopes saved to: {output_csv}")

# --- 10) Plot slope vs channel number ---
plt.figure(figsize=(12, 6))
plt.plot(df['Channel'], df['slope_deg'], linestyle='-')
plt.xlabel('Channel')
plt.ylabel('Slope (degrees)')
plt.title('Slope by channel')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:

extent = [0, 20, 0, 10]

# Create the figure and axis
fig, ax = plt.subplots(figsize=(12, 8))
im = ax.imshow(
    np.ma.masked_invalid(slope_crop),
    cmap='terrain',
    extent=extent
)

# Adjust the colorbar size
cbar = fig.colorbar(im, ax=ax, shrink=0.65, aspect=20, pad = 0.05)  # Control size
cbar.set_label('Pendiente (°)')

ax.set_title('Slope map')
plt.tight_layout()
plt.savefig('/Users/veronicaage/Downloads/slopemap.png', dpi=300)

plt.show()

In [ ]:
from matplotlib.colors import LightSource

# 1) Get pixel size from your output transform
dx = abs(out_trans.a)   # pixel width in meters
dy = abs(out_trans.e)   # pixel height in meters

# 2) Compute hillshade from the DEM crop
ls    = LightSource(azdeg=315, altdeg=45)
shade = ls.hillshade(dem_crop, vert_exag=1, dx=dx, dy=dy)

# 3) Plot hillshade + overlay slope
plt.figure(figsize=(8, 6))
plt.imshow(shade, cmap='gray', extent=extent)
plt.imshow(
    np.ma.masked_invalid(slope_crop),
    cmap='terrain',
    extent=extent,
    alpha=0.6
)
plt.colorbar(label='Slope (°)')
plt.title('Hillshade + Slope overlay')
plt.xlabel('X (EPSG:2927)')
plt.ylabel('Y (EPSG:2927)')
plt.tight_layout()
plt.show()


In [ ]:
# — assuming you already have dem_crop, slope_crop, out_trans, df, x_pts, y_pts, slope_pts —

# 1) Hillshade Calculation
dx = abs(out_trans.a)
dy = abs(out_trans.e)
ls = LightSource(azdeg=315, altdeg=45)
#shade = ls.hillshade(dem_crop, vert_exag=1, dx=dx, dy=dy)

# 2) Sorting Points for Cable Line
df_line = df.copy().sort_values('Channel')
x_line = transformer.transform(df_line['Longitude [°]'].values,
                               df_line['Latitude [°]'].values)[0]
y_line = transformer.transform(df_line['Longitude [°]'].values,
                               df_line['Latitude [°]'].values)[1]

# 3) Make sure the extent matches the range of the data
x_min, x_max = min(x_line), max(x_line)
y_min, y_max = min(y_line), max(y_line)
extent = [x_min, x_max, y_min, y_max]

# 4) Create the figure and main axis
fig, ax = plt.subplots(figsize=(12, 8))

# 5) Plot the hillshade (background) with adjusted extent
#ax.imshow(shade, cmap='gray', extent=extent, alpha=0.7)

# 6) Overlay slope map with transparency
im = ax.imshow(
    np.ma.masked_invalid(slope_crop),
    cmap='terrain',
    extent=extent,
    alpha=0.6
)



# 8) Scatter plot of points with slope values
sc = ax.scatter(
    x_pts, y_pts,
    c=slope_pts, cmap='viridis',
    edgecolor='black', s=5, zorder=3,
    #label='Slope @ points'
)

#7) Overlay cable path
ax.plot(
    x_line, y_line,
    '-', color='black',
    linewidth=1, label='Cable Path'
)

# 9) Adjust the colorbar size
cbar = fig.colorbar(im, ax=ax, shrink=0.5, aspect=20, pad=0.05)
cbar.set_label('Slope (°)')

# 10) Labels, title, and legend
#ax.set_title('Cable Overlay on Hillshade + Slope Map', fontsize=16)
ax.set_xlabel('X (EPSG:2927)', fontsize=14)
ax.set_ylabel('Y (EPSG:2927)', fontsize=14)
ax.legend(loc='lower right', fontsize=12)

plt.tight_layout()
plt.savefig('/Users/veronicaage/Downloads/cable_lidar.png', dpi=300)
plt.show()
